In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('C://Users/Антон/Desktop/web_2/data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision



def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации   
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    df = items_weights.copy()
    df = df.sort_values('weight',ascending=False)
    r = np.random.randint(n,len(items_weights))
    df= df.iloc[:r]
    recs = np.random.choice(df.iloc[:r,0],size = n,replace=False)
    
    return recs.tolist()

In [9]:
# потестируем  алгоритм
test = pd.DataFrame({'item_id':[1,2,3,4,5,6,7,8,9,10],'weight':[0.3,0.25,0.2,0.15,0.1,0.08,0.07,0.05,0.003,0.001]})

In [10]:
weighted_random_recommendation(test)

[1, 6, 3, 4, 2]

In [11]:
d={}
for i in range(1,11):
    d[i] = 0
    
for i in range(10000):
    a = weighted_random_recommendation(test,n=3)
    for j in a:
        d[j]+=1
d

{1: 5617,
 2: 5730,
 3: 5651,
 4: 4327,
 5: 3219,
 6: 2365,
 7: 1656,
 8: 982,
 9: 453,
 10: 0}

In [12]:
#вес по сумме продаж

items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] =  np.log(items_weights.sales_value+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value',axis=1,inplace=True)

In [13]:
items_weights.weight.sum()

1.0

In [14]:
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000006
min,2.567100e+04,0.000003
25%,9.665830e+05,0.000007
50%,1.448516e+06,0.000010
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000050


In [ ]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights=item_weights, n=5))

In [15]:
weighted_random_recommendation(items_weights)

[10149439, 1060926, 1042564, 936858, 826151]

In [16]:
#вес по объему продаж

items_weights = data.groupby('item_id')['quantity'].sum().reset_index()
items_weights['weight'] =  np.log(items_weights.quantity+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('quantity',axis=1,inplace=True)

In [17]:
items_weights.weight.sum()

0.9999999999999999

In [18]:
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000007
min,2.567100e+04,0.000004
25%,9.665830e+05,0.000006
50%,1.448516e+06,0.000008
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000098


In [19]:
weighted_random_recommendation(items_weights)

[10312180, 2041688, 996506, 13525069, 15596923]

In [20]:
#вес по количеству продаж 

items_weights = data.groupby('item_id')['sales_value'].count().reset_index()
items_weights['weight'] =  np.log(items_weights.sales_value+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value',axis=1,inplace=True)

In [21]:
items_weights.weight.sum()

1.0000000000000002

In [22]:
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000007
min,2.567100e+04,0.000006
25%,9.665830e+05,0.000006
50%,1.448516e+06,0.000009
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000055


In [23]:
weighted_random_recommendation(items_weights)

[12949759, 13095719, 13944877, 955206, 15717312]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [25]:
# your_code


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code